In [1]:
import os
import librosa
import numpy as np
from sklearn.preprocessing import StandardScaler
import pickle

def extract_combined_features(file_path, sr=44100, n_mfcc=13, hop_length=512, n_fft=2048):
    y, sr = librosa.load(file_path, sr=sr)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length, n_fft=n_fft)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr, hop_length=hop_length, n_fft=n_fft)
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr, hop_length=hop_length, n_fft=n_fft)
    tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(y), sr=sr)
    features = np.concatenate((mfccs, chroma, spectral_contrast, tonnetz), axis=0)
    return features

# Base directory containing your class folders
base_dir = '/Users/danielporras/Musica/musica_analysis/music_files/3_class_audio extraction'
class_folders = os.listdir(base_dir)

features = []
labels = []

# Iterate over class folders and extract features
for label, folder_name in enumerate(class_folders):
    folder_path = os.path.join(base_dir, folder_name)
    if os.path.isdir(folder_path):
        for file_name in os.listdir(folder_path):
            if file_name.endswith('.mp3') or file_name.endswith('.wav') or file_name.endswith('.m4a'):
                file_path = os.path.join(folder_path, file_name)
                try:
                    file_features = extract_combined_features(file_path)
                    features.append(file_features)
                    labels.append(label)
                except Exception as e:
                    print(f"Error processing {file_name}: {e}")

# Convert lists to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Reshape features for CNN input and normalize
features = features.reshape(features.shape[0], features.shape[1], features.shape[2], 1)
scaler = StandardScaler()
features_flattened = features.reshape(features.shape[0], -1)
features_normalized = scaler.fit_transform(features_flattened)
features_normalized = features_normalized.reshape(features.shape)

# Save the scaler for later use
with open('scaler_2.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# At this point, features_normalized contains your processed features ready for CNN input
# and labels contains the corresponding class labels for each feature set.


/var/folders/_t/p3lht79d2_jd6gwjl515j1g40000gn/T/ipykernel_23066/3000728180.py:8: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, sr=sr)
/Users/danielporras/Musica/new_env_3_10/lib/python3.10/site-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


KeyboardInterrupt: 